In [1]:
from app.Transformer_Classifier import Transformer_Classifier 
from app.Data_Loader import Data_Loader
from app.TextRank_Extractor import TextRank_Extractor
from app.Keyword_Classifier import Keyword_Classifier
from app.common.MySQLUtility import MySQLUtility
import os 

domains = ['esg', 'liabilities' ] #'liabilities', 'esg'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = './store/genuine-wording-key.json'

DB_HOST = '34.170.168.203'
DB_USER = 'root'
DB_PASSWORD = 'nu123456'
DB_NAME = 'lca_db'

class Data_ETL_Pipeline(object):
    dbutil = None
    data_load = None
    textrank = None 
    key_classifier = None
    class_service = None

    def __init__(self):
        self.dbutil = MySQLUtility(DB_HOST, DB_USER, DB_PASSWORD, DB_NAME)
        self.data_load = Data_Loader(self.dbutil)
        self.textrank = TextRank_Extractor(self.dbutil)
        self.key_classifier = Keyword_Classifier(self.dbutil)
        self.class_service = Transformer_Classifier(self.dbutil)
        pass    

    def create_dataset(self):
        print("dbutil.db_cleanup():")
        self.dbutil.clean_db()
        print("dbutil.create_database():")
        self.dbutil.create_database() 

    def load_seed_training_data(self):
        print("data_load.import_seed_data_batch():")
        self.data_load.import_seed_data_batch()

        for domain in domains:
            print("textrank.extract_keyword_seed_data():" + domain)
            self.textrank.extract_keyword_seed_data(domain) 

            print("textrank.load_seed_to_training_data_batch():" + domain)
            self.data_load.load_seed_to_training_data_batch(domain) 
    
    def load_contract_data(self):
        for domain in domains:
            print("self.data_load.import_reports_contract_data()" + domain)
            self.data_load.import_reports_contract_data(domain)

    def process_keyword_model(self):
        for domain in domains:
            print("key_classifier.prepare_training_data():" + domain)
            self.key_classifier.prepare_training_data(domain)

            print("key_classifier.train_model():" + domain)
            self.key_classifier.train_model(domain)

            print("key_classifier.evaluate_model():" + domain)
            self.key_classifier.evaluate_model(domain)

            print("key_classifier.process_contract_data():" + domain)
            self.key_classifier.process_contract_data(domain)

    def process_transformer_model(self):
        for domain in domains:
            print("class_service.training():" + domain)
            self.class_service.training(domain)    

            print("class_service.process_contract_training_data_eval():" + domain)
            self.class_service.process_contract_training_data_eval(domain)

    def evaluate_results(self):
        for domain in domains:
            print ("key_classifier.Keyword Classifier Accuracy: " + domain)
            self.key_classifier.evaluate_model(domain) 
            
            print ("class_service.Transformer Classifier Accuracy: " + domain)
            self.class_service.evalute_model(domain)

if __name__ == '__main__': 
    dbloader = Data_ETL_Pipeline()
    dbloader.create_dataset()
    dbloader.load_seed_training_data() 
    dbloader.load_contract_data()
    dbloader.process_keyword_model()
    dbloader.process_transformer_model()
    dbloader.evaluate_results()


/Users/saurabhkaushik/Workspace/lca-ai-services/env/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


dbutil.db_cleanup():
DB Pool Created.
Table contract_data successfully deleted.
Table seed_data successfully deleted.
Table training_data successfully deleted.
dbutil.create_database():
Table contract_data successfully created.
Table seed_data successfully created.
Table training_data successfully created.
data_load.import_seed_data_batch():
Working with  seed_data ESG v2.0.csv
INSERT INTO seed_data (id, created, keywords, content, type, label, domain, userid) VALUES ('372f7a47-a608-4fb0-b977-3a1c1198d210', '2022-12-01 06:31:02', 'governance, transparency, diversity, procedure, company, key environmental social policy, disclosure, ethical conduct, process, corporate governance structure, risk management', 'Governance: corporate governance structures and processes by which companies are directed and controlled (e.g. board structure and diversity, ethical conduct, risk management, disclosure and transparency), including the governance of key environmental and social policies and procedur

ERROR:root:Traceback (most recent call last):
  File "/Users/saurabhkaushik/Workspace/lca-ai-services/app/Data_Loader.py", line 25, in import_reports_contract_data
    filelist = os.listdir(reports_folder)
FileNotFoundError: [Errno 2] No such file or directory: './data/reports/liabilities/'



1 record(s) affected
self.data_load.import_reports_contract_data()liabilities
key_classifier.prepare_training_data():esg
SELECT * from seed_data where domain='esg';
0      environment
1      environment
2      environment
3      environment
4      environment
          ...     
268     governance
269     governance
270     governance
271     governance
272     governance
Name: label, Length: 273, dtype: object
key_classifier.train_model():esg
key_classifier.evaluate_model():esg
SELECT * from seed_data where domain='esg';
0      environment
1      environment
2      environment
3      environment
4      environment
          ...     
268     governance
269     governance
270     governance
271     governance
272     governance
Name: label, Length: 273, dtype: object
Classification Report : 
               precision    recall  f1-score   support

 environment       1.00      0.99      0.99        83
  governance       0.97      0.87      0.92        95
      social       0.88      0.97  

ERROR:root:Traceback (most recent call last):
  File "/Users/saurabhkaushik/Workspace/lca-ai-services/app/Keyword_Classifier.py", line 53, in train_model
    self.trainDF['text'], self.trainDF['label'])
  File "/Users/saurabhkaushik/Workspace/lca-ai-services/env/lib/python3.7/site-packages/sklearn/pipeline.py", line 390, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "/Users/saurabhkaushik/Workspace/lca-ai-services/env/lib/python3.7/site-packages/sklearn/pipeline.py", line 355, in _fit
    **fit_params_steps[name],
  File "/Users/saurabhkaushik/Workspace/lca-ai-services/env/lib/python3.7/site-packages/joblib/memory.py", line 349, in __call__
    return self.func(*args, **kwargs)
  File "/Users/saurabhkaushik/Workspace/lca-ai-services/env/lib/python3.7/site-packages/sklearn/pipeline.py", line 893, in _fit_transform_one
    res = transformer.fit_transform(X, y, **fit_params)
  File "/Users/saurabhkaushik/Workspace/lca-ai-services/env/lib/python3.7/site-packages/sklearn/featur

...

Query:  SELECT * from training_data where domain='liabilities';
SELECT * from training_data where domain='liabilities';
0
None
-1 record(s) affected
key_classifier.Keyword Classifier Accuracy: esg
SELECT * from seed_data where domain='esg';
0      environment
1      environment
2      environment
3      environment
4      environment
          ...     
268     governance
269     governance
270     governance
271     governance
272     governance
Name: label, Length: 273, dtype: object
Classification Report : 
               precision    recall  f1-score   support

 environment       1.00      0.99      0.99        83
  governance       0.97      0.87      0.92        95
      social       0.88      0.97      0.92        95

    accuracy                           0.94       273
   macro avg       0.95      0.94      0.94       273
weighted avg       0.94      0.94      0.94       273

Confusion Matrix: 
 [[82  0  1]
 [ 0 83 12]
 [ 0  3 92]]
Accuracy Score: 
 94.13919413919413 %
class_se

/Users/saurabhkaushik/Workspace/lca-ai-services/env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/saurabhkaushik/Workspace/lca-ai-services/env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/saurabhkaushik/Workspace/lca-ai-services/env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

Classification Report : 
               precision    recall  f1-score   support

                   0.00      0.00      0.00         0
 environment       0.99      0.95      0.97       130
  governance       0.95      0.91      0.93       135
      social       0.92      0.96      0.94       164

    accuracy                           0.94       429
   macro avg       0.72      0.71      0.71       429
weighted avg       0.95      0.94      0.95       429

Confusion Matrix: 
 [[  0   0   0   0]
 [  3 124   0   3]
 [  1   0 123  11]
 [  0   1   6 157]]
Accuracy Score: 
 94.17249417249417 %
key_classifier.Keyword Classifier Accuracy: liabilities
SELECT * from seed_data where domain='liabilities';


ERROR:root:Traceback (most recent call last):
  File "/Users/saurabhkaushik/Workspace/lca-ai-services/app/Keyword_Classifier.py", line 67, in evaluate_model
    predicted = self.text_clf.predict(self.trainDF['text'])
  File "/Users/saurabhkaushik/Workspace/lca-ai-services/env/lib/python3.7/site-packages/sklearn/utils/metaestimators.py", line 113, in <lambda>
    out = lambda *args, **kwargs: self.fn(obj, *args, **kwargs)  # noqa
  File "/Users/saurabhkaushik/Workspace/lca-ai-services/env/lib/python3.7/site-packages/sklearn/pipeline.py", line 469, in predict
    Xt = transform.transform(Xt)
  File "/Users/saurabhkaushik/Workspace/lca-ai-services/env/lib/python3.7/site-packages/sklearn/feature_extraction/text.py", line 1661, in transform
    X, accept_sparse="csr", dtype=FLOAT_DTYPES, copy=copy, reset=False
  File "/Users/saurabhkaushik/Workspace/lca-ai-services/env/lib/python3.7/site-packages/sklearn/base.py", line 566, in _validate_data
    X = check_array(X, **check_params)
  File "/U

Series([], Name: label, dtype: float64)
class_service.Transformer Classifier Accuracy: liabilities
Query:  SELECT * from training_data where domain='liabilities';
SELECT * from training_data where domain='liabilities';



In [2]:
from app.common.GCP_Storage import GCP_Storage

import os 
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = './store/genuine-wording-key.json'

domains = ['liabilities', 'esg']
loader = GCP_Storage(domains)

#loader.setup_bucket()
#loader.upload_models()
#loader.download_models()
#loader.download_seed_data()
#loader.upload_seed_data()